# Link features

So far, we haven't enabled ourselves to query for the features[^features] of ingested data, and used LaminDB like a data lake.

[^features]: We'll mostly use the term feature for synonyms variable (statistics), column and field (databases), dimension (machine learning).


We can also use LaminDB like a queryable data warehouse to store links[^relations] and monitor data integrity.

Let us explain how to implement this by providing feature models at ingestion!

[^relations]: We mostly use the term link for synonyms relations and references.

In [1]:
import lamindb as db
import bionty as bt  # https://lamin.ai/docs/bionty
import scanpy as sc  # https://scanpy.readthedocs.io

db.header()

2022-08-23 11:47:36,558:INFO - Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2022-08-23 11:47:36,559:INFO - NumExpr defaulting to 8 threads.


id,ZKJX7AnXzQQp
version,draft
time_init,2022-07-30 16:14
time_run,2022-08-23 09:47
pypackage,bionty==0.0.6+17.g0499eed lamindb==0.2.1 scanpy==1.8.2


## Example datasets

Consider
- `data1`: a flow cytometry dataset in form of an `.fcs` file
- `data2`: a scRNA-seq count matrix in form of an `AnnData` object in memory

In [2]:
data1 = db.datasets.file_fcs()
data1

PosixPath('example.fcs')

In [3]:
data2 = sc.read(db.datasets.file_10x_mouse_sc_E18_brain())

## Define feature models

For `data1`, we specify a feature model using the `bionty` `Gene` entity with gene symbols.

In [5]:
feature_model1 = bt.Gene(id=bt.lookup.gene_id.name, species=bt.lookup.species.human)

Let us now ingest the data by passing a feature model to `db.do.ingest.add`, which will enable us to query the `dobject` by features by creating all necessary links in the background.

It will also log out and store information on data integrity:

In [6]:
db.do.ingest.add(data1, feature_model=feature_model1)

🔶 name column not found, using index as features.


/opt/miniconda3/envs/py39/lib/python3.9/site-packages/fcsparser/api.py:440: UserWarning: The default channel names (defined by the $PnS parameter in the FCS file) were not unique. To avoid problems in downstream analysis, the channel names have been switched to the alternate channel names defined in the FCS file. To avoid seeing this warning message, explicitly instruct the FCS parser to use the alternate channel names by specifying the channel_naming parameter.
  warnings.warn(msg)
/opt/miniconda3/envs/py39/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


✅ 9 terms (56.2%) are linked.
🔶 7 terms (43.8%) are not linked.


Using this feature model, we can't link and hence won't be able to query for 9 features.

We can overcome this by working with a custom feature model, discussed later.

Features in data2 are indexed by Ensemble gene ids. For an overview of gene ids, see: [`bt.lookup.gene_id`](https://lamin.ai/docs/bionty/api).

In [7]:
data2.var.head()

,gene_name,feature_types,genome
gene_ids,,,
ENSMUSG00000002459,Rgs20,Gene Expression,mm10
ENSMUSG00000085623,Gm16041,Gene Expression,mm10
ENSMUSG00000061024,Rrs1,Gene Expression,mm10
ENSMUSG00000025911,Adhfe1,Gene Expression,mm10
ENSMUSG00000099827,Gm29520,Gene Expression,mm10


Hence, we use a feature model based on Ensemble IDs and ingest the data with it.

In [8]:
feature_model2 = bt.Gene(
    id=bt.lookup.gene_id.ensembl_gene_id, species=bt.lookup.species.mouse
)

In [9]:
db.do.ingest.add(data2, name="10x_mouse_sc_E18_brain", feature_model=feature_model2)

🔶 ensembl_gene_id column not found, using index as features.
✅ 4933 terms (98.7%) are linked.
🔶 67 terms (1.3%) are not linked.


We can retrieve the integrity information through `.logs`:

In [10]:
db.do.ingest.logs

{'example.fcs': {'feature': 'name',
  'n_mapped': 9,
  'percent_mapped': 56.2,
  'unmapped': Index(['FSC-A', 'FSC-H', 'SSC-A', 'KI67', 'CD3', 'CD45RO', 'VIVID / CD14'], dtype='object')},
 '10x_mouse_sc_E18_brai.h5ad': {'feature': 'ensembl_gene_id',
  'n_mapped': 4933,
  'percent_mapped': 98.7,
  'unmapped': Index(['ENSMUSG00000097285', 'ENSMUSG00000097454', 'ENSMUSG00000097045',
         'ENSMUSG00000104367', 'ENSMUSG00000097774', 'ENSMUSG00000097872',
         'ENSMUSG00000115956', 'ENSMUSG00000097868', 'ENSMUSG00000075024',
         'ENSMUSG00000086879', 'ENSMUSG00000094351', 'ENSMUSG00000096919',
         'ENSMUSG00000116028', 'ENSMUSG00000097094', 'ENSMUSG00000097019',
         'ENSMUSG00000097459', 'ENSMUSG00000097005', 'ENSMUSG00000097249',
         'ENSMUSG00000051219', 'ENSMUSG00000092252', 'ENSMUSG00000097213',
         'ENSMUSG00000097364', 'ENSMUSG00000097138', 'ENSMUSG00000096931',
         'ENSMUSG00000074402', 'ENSMUSG00000066723', 'ENSMUSG00000097864',
         'ENSMUSG0

Finalize the ingestion.

In [ ]:
db.do.ingest.commit()

✅ Annotated data mWmIx2pzpzz4eCOa0jR5f with the following features:
+------------+------------+------------+
| geneset.id | biometa.id | species.id |
+------------+------------+------------+
|     3      |     3      |     1      |
+------------+------------+------------+
✅ Annotated data ZbdKAJ3DAEszfn8MpGCwd with the following features:
+------------+------------+------------+
| geneset.id | biometa.id | species.id |
+------------+------------+------------+
|     4      |     4      |     2      |
+------------+------------+------------+
✅ Ingested the following dobjects:
+-------------------------------------------------+-----------------------------------+-----------------------+
|                     dobject                     |              jupynb               |         user          |
+-------------------------------------------------+-----------------------------------+-----------------------+
|     example.fcs (mWmIx2pzpzz4eCOa0jR5f, 1)      | 'Link features' (ZKJX7AnXzQQp, 

RuntimeError: Make sure you save the notebook in your editor before publishing!
You can avoid the need for manually saving in Jupyter Lab, which auto-saves the buffer during publish.